# Algotrading 

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd

from surfingcrypto import Config,TS
from surfingcrypto.algotrading.backtesting import BackTest
from surfingcrypto.algotrading.features import BinaryLaggedLogReturns
from surfingcrypto.algotrading.model import Model
from surfingcrypto.reporting.figures import BacktestPerformancePlot
from surfingcrypto.reporting.reporting import report_tomorrow_prediction


/opt/miniconda3/envs/cryptoenv/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [2]:
coins={
    "BTC":"",
    "ETH":"",
}
c = Config(coins,"/Users/giorgiocaizzi/Documents/GitHub/surfingcrypto/config")


In [3]:
ticker = "ETH"
ts = TS(c, coin=ticker)
ts.ta_indicators()


## Feature Engineering

The chosen algo uses binary encoding for upward or downward movements.

In [4]:
f = BinaryLaggedLogReturns(
    [1, 2, 3, 7, 14],
    ts,
    indicators=[
        "SMA_12_26_Signal",
        "SMA_100_200_Signal",
        "MACD_12_26_9_Signal",
        "BB_20_2_Signal",
        "RSI_14_Signal",
    ],
)
f


BinaryLaggedLogReturns(ts=ETH,lags=[1, 2, 3, 7, 14])

In [5]:
f.indicators

{'i_01': 'SMA_12_26_Signal',
 'i_02': 'SMA_100_200_Signal',
 'i_03': 'MACD_12_26_9_Signal',
 'i_04': 'BB_20_2_Signal',
 'i_05': 'RSI_14_Signal'}

### Signals

- SMA Cross
- MACD
- Bollinger Bands
- RSI

In [6]:
f.df.tail()


,Open,High,Low,Close,Volume,Market Cap,SMA_26,SMA_12,SMA_200,SMA_100,...,SMA_12_26_Signal,SMA_100_200_Signal,MACD_12_26_9_Signal,PREV_STOCK,PREV_LOWERBB,PREV_UPPERBB,BBL_20_2_Signal,BB_20_2_Signal,PREV_RSI,RSI_14_Signal
Date,,,,,,,,,,,,,,,,,,,,,
2022-05-09 00:00:00+00:00,2391.238078,2403.478519,2120.092136,2125.672740,3.439575e+10,2.565979e+11,2689.265506,2587.535339,3038.946438,2653.703775,...,0.0,0.0,0.0,2390.242663,2457.013690,2918.524709,0.0,0.0,35.425652,1.0
2022-05-10 00:00:00+00:00,2123.040831,2320.823379,2089.052937,2225.735405,3.628221e+10,2.687063e+11,2667.543130,2540.003732,3033.026995,2652.616469,...,0.0,0.0,0.0,2125.672740,2328.136460,2972.432101,0.0,0.0,28.261553,1.0
2022-05-11 00:00:00+00:00,2225.016794,2313.733229,1920.234516,1971.188698,4.351551e+10,2.380018e+11,2635.182502,2481.735076,3024.966499,2648.388399,...,0.0,0.0,0.0,2225.735405,2260.367972,2979.105683,0.0,0.0,33.720853,1.0
2022-05-12 00:00:00+00:00,1971.565315,2061.724537,1671.830539,1890.164211,4.091497e+10,2.282437e+11,2598.944498,2423.465259,3016.856956,2642.522174,...,0.0,0.0,0.0,1971.188698,2129.538834,3031.321770,0.0,0.0,27.903518,1.0
2022-05-13 00:00:00+00:00,1888.642758,2058.374253,1870.895728,1934.727849,2.383437e+10,2.336508e+11,2566.887510,2361.012121,3008.363883,2638.126651,...,0.0,0.0,0.0,1890.164211,2003.119974,3071.896630,0.0,0.0,26.345524,1.0


### Model dataframe

In [7]:
f.model_df.head()

,ETH,i_01,i_02,i_03,i_04,i_05,returns,direction,i_01_lag01,i_01_lag02,...,i_04_lag01,i_04_lag02,i_04_lag03,i_04_lag07,i_04_lag14,i_05_lag01,i_05_lag02,i_05_lag03,i_05_lag07,i_05_lag14
Date,,,,,,,,,,,,,,,,,,,,,
2018-05-02 00:00:00+00:00,575.020073,1.0,1.0,1.0,1.0,0.0,0.023237,1,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-05-03 00:00:00+00:00,650.269461,1.0,1.0,1.0,1.0,0.0,0.122982,1,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-05-04 00:00:00+00:00,656.986138,1.0,1.0,1.0,1.0,1.0,0.010276,1,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-05-05 00:00:00+00:00,682.488706,1.0,1.0,1.0,1.0,1.0,0.038083,1,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2018-05-06 00:00:00+00:00,662.855328,1.0,1.0,1.0,1.0,1.0,-0.029189,-1,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


### Build and Apply Classification Machine Learning Algorithms

Select Logistic regression, Gaussian Naive Bayes, Support Vector Machine, Random Forest, and MLP Classifier approaches to predict the market direction. Please refer sklearn documentation for detail on these and other algorithms.
https://scikit-learn.org/stable/index.html

In [8]:
models = [
    "log_reg",
    "gauss_nb",
    "svm",
    "random_forest",
    "MLP",
]
models=[Model(x,f) for x in models]

/opt/miniconda3/envs/cryptoenv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


### Strategy evalutation using Backtrader

Strategy: 
1.	Buy when the predicted value is +1 and sell (only if stock is in possession) when the predicted value is -1.
2.	All-in strategy—when creating a buy order, buy as many shares as possible.
3.	Short selling is not allowed

In [9]:
start = "2021-01-01"

performance = {}
backtests = {}
for model in models:
    # run
    i_backtest = BackTest(model,start, verbose=False)
    i_backtest.run()
    backtests[model.name] = i_backtest
    performance[model.name] = i_backtest.performance_stats()
performance = pd.DataFrame(performance)



In [10]:
performance


,log_reg,gauss_nb,svm,random_forest,MLP
Annual return,1.718876,2.071350,2.523996,1.314962,1.078200
Cumulative returns,6.218322,8.184501,11.051641,4.253008,3.244345
Annual volatility,0.709847,0.750160,0.630071,0.603280,0.535666
Sharpe ratio,1.761039,1.874352,2.314857,1.691317,1.630968
Calmar ratio,3.789346,3.712070,7.990818,3.187114,3.286222
Stability,0.842553,0.895681,0.823255,0.489353,0.644784
Max drawdown,-0.453608,-0.558004,-0.315862,-0.412587,-0.328097
Omega ratio,1.436424,1.459024,1.638278,1.467232,1.497412
Sortino ratio,2.888695,2.933825,3.903499,2.811512,2.766190
Skew,0.548679,0.050052,0.453880,0.547543,0.820636


## Choose best model from backtesting results

Choose the one with the greates cumulative returns.

In [11]:
best_model=[x for x in models if x.name==performance.loc["Cumulative returns"].idxmax()][0]
backtest = backtests[performance.loc["Cumulative returns"].idxmax()]

best_model


Model(name=svm)

In [12]:
backtest.end_value


15064.550878998918

In [13]:
backtest.print_log()


2021-01-02, Open: 600.888, Close: 638.266
2021-01-02,     BUY CREATED --- Size: 2.060, Price: 600.860, Cost: 1237.500
2021-01-02,     BUY EXECUTED --- Size: 2.060, Price: 600.888, Cost: 1237.559, Commission: 12.376
2021-02-20, Open: 1617.316, Close: 1584.003
2021-02-20,     SELL CREATED --- Size: -2.060, Price: 1617.532, Cost: -3331.387
2021-02-20,     SELL EXECUTED --- Size: -2.060, Price: 1617.316, Cost: -3330.942, Commission: 33.309
2021-02-20,          -> OPERATION RESULT --- Gross: 2093.383, Net: 2047.698
2021-02-22, Open: 1596.353, Close: 1464.957
2021-02-22,     BUY CREATED --- Size: 2.045, Price: 1596.389, Cost: 3264.721
2021-02-22,     BUY EXECUTED --- Size: 2.045, Price: 1596.353, Cost: 3264.648, Commission: 32.646
2021-02-23, Open: 1464.477, Close: 1292.240
2021-02-23,     SELL CREATED --- Size: -2.045, Price: 1464.957, Cost: -2995.933
2021-02-23,     SELL EXECUTED --- Size: -2.045, Price: 1464.477, Cost: -2994.952, Commission: 29.950
2021-02-23,          -> OPERATION RESULT

# Plotting

In [14]:
backtest.cerebro.plot(
            style="candlesticks",
            barup="darkgreen",
            bardown="darkred",
            numfigs=2,
            iplot=False,
            fmt_x_ticks="%Y-%b-%d",
        )


[[<Figure size 1280x960 with 4 Axes>, <Figure size 1280x960 with 4 Axes>]]

In [15]:
btp=BacktestPerformancePlot(backtest)

# Apply to future

In [16]:
best_model.feature.get_future_x()

i_01_lag01    0.0
i_01_lag02    0.0
i_01_lag03    0.0
i_01_lag07    0.0
i_01_lag14    0.0
i_02_lag01    0.0
i_02_lag02    0.0
i_02_lag03    0.0
i_02_lag07    0.0
i_02_lag14    0.0
i_03_lag01    0.0
i_03_lag02    0.0
i_03_lag03    0.0
i_03_lag07    0.0
i_03_lag14    0.0
i_04_lag01    0.0
i_04_lag02    0.0
i_04_lag03    0.0
i_04_lag07    0.0
i_04_lag14    0.0
i_05_lag01    1.0
i_05_lag02    1.0
i_05_lag03    1.0
i_05_lag07    1.0
i_05_lag14    1.0
Name: 2022-05-14 00:00:00+00:00, dtype: float64

In [17]:
report_tomorrow_prediction(best_model.make_tomorrow_prediction())

/opt/miniconda3/envs/cryptoenv/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  "X does not have valid feature names, but"


'SELL / STAY OUT OF MARKET'